In [ ]:
# !pip install deepface
# !pip install opencv-python
# !pip install scikit-image
# !pip install onnxruntime insightface


In [1]:
import cv2
import numpy as np
from skimage import io


In [18]:
alg = "haarcascade_frontalface_default.xml"
haar_cascade = cv2.CascadeClassifier(alg)
file_name = "data/955px-NASA_Astronaut_Group_18.jpg"

img = cv2.imread(file_name, 0)
grey_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

faces = haar_cascade.detectMultiScale(grey_img, scaleFactor=1.15, minNeighbors=3, minSize=(60, 60))


In [ ]:
faces.shape

In [8]:
i = 0

for x,y,w,h in faces:
    cropped_image = img[y:y+h, x: x+w]
    target_file_name = 'stored_faces/' + str(i) + '.jpg'
    cv2.imwrite(target_file_name, cropped_image)
    i = i+1
    


Some reference  
YuNet
https://medium.com/pythons-gurus/what-is-the-best-face-detector-ab650d8c1225  
https://learnopencv.com/what-is-face-detection-the-ultimate-guide/  


In [ ]:
!wget https://github.com/astaileyyoung/CineFace/raw/main/research/data/face_detection_yunet_2023mar.onnx -P ./model/



In [19]:
class FaceDetectorYunet():
    def __init__(self, model_path='./face_detection_yunet_2023mar.onnx',
                 img_size=(400, 400),
                 threshold=0.5):
        self.model_path = model_path
        self.img_size = img_size
        self.fd = cv2.FaceDetectorYN_create(str(model_path),
                                            "",
                                             img_size,
                                             score_threshold=threshold)


    def draw_faces(self,
                   image,
                   faces,
                   draw_landmarks=False,
                   show_confidence=False):
        for face in faces:
            color = (0, 0, 255)
            thickness = 2
            cv2.rectangle(image, (face['x1'], face['y1']), (face['x2'], face['y2']), color, thickness, cv2.LINE_AA)

            if draw_landmarks:
                landmarks = face['landmarks']
                for landmark in landmarks:
                    radius = 5
                    thickness = -1
                    cv2.circle(image, landmark, radius, color, thickness, cv2.LINE_AA)

            if show_confidence:
                confidence = face['confidence']
                confidence = "{:.2f}".format(confidence)
                position = (face['x1'], face['y1'] - 10)
                font = cv2.FONT_HERSHEY_SIMPLEX
                scale = 0.5
                thickness = 2
                cv2.putText(image, confidence, position, font, scale, color, thickness, cv2.LINE_AA)
        return image

    def scale_coords(self, image, prediction):
        ih, iw = image.shape[:2]
        rw, rh = self.img_size
        a = np.array([
                (prediction['x1'], prediction['y1']),
                (prediction['x1'] + prediction['x2'], prediction['y1'] + prediction['y2'])
                    ])
        b = np.array([iw/rw, ih/rh])
        c = a * b
        prediction['img_width'] = iw
        prediction['img_height'] = ih
        prediction['x1'] = int(c[0,0].round())
        prediction['x2'] = int(c[1,0].round())
        prediction['y1'] = int(c[0,1].round())
        prediction['y2'] = int(c[1,1].round())
        prediction['face_width'] = (c[1,0] - c[0,0])
        prediction['face_height'] = (c[1,1] - c[0,1])
        # prediction['face_width'] = prediction['x2'] - prediction['x1']
        # prediction['face_height'] = prediction['y2'] - prediction['y1']
        prediction['area'] = prediction['face_width'] * prediction['face_height']
        prediction['pct_of_frame'] = prediction['area']/(prediction['img_width'] * prediction['img_height'])
        return prediction

    def detect(self, image):
        if isinstance(image, str):
            image = cv2.imread(str(image))
        gray_img = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
        img = cv2.resize(gray_img, self.img_size)
        self.fd.setInputSize(self.img_size)
        _, faces = self.fd.detect(img)
        if faces is None:
            return None
        else:
            predictions = self.parse_predictions(image, faces)
            return predictions

    def parse_predictions(self, image, faces):
        data = []
        for num, face in enumerate(list(faces)):
            x1, y1, x2, y2 = list(map(int, face[:4]))
            landmarks = list(map(int, face[4:len(face)-1]))
            landmarks = np.array_split(landmarks, len(landmarks) / 2)
            positions = ['left_eye', 'right_eye', 'nose', 'right_mouth', 'left_mouth']
            landmarks = {positions[num]: x.tolist() for num, x in enumerate(landmarks)}
            confidence = face[-1]
            datum = {'x1': x1,
            'y1': y1,
            'x2': x2,
            'y2': y2,
            'face_num': num,
            'landmarks': landmarks,
            'confidence': confidence,
            'model': 'yunet'}
        d = self.scale_coords(image, datum)
        data.append(d)
        return data


In [20]:
class FaceDetectorYunet():
    def __init__(self,
                  model_path='./face_detection_yunet_2023mar.onnx',
                  img_size=(300, 300),
                  threshold=0.5):
        self.model_path = model_path
        self.img_size = img_size
        self.fd = cv2.FaceDetectorYN_create(str(model_path),
                                            "",
                                            img_size,
                                            score_threshold=threshold)

    def draw_faces(self,
                   image,
                   faces,
                   draw_landmarks=False,
                   show_confidence=False):
        for face in faces:
            color = (0, 0, 255)
            thickness = 2
            cv2.rectangle(image, (face['x1'], face['y1']), (face['x2'], face['y2']), color, thickness, cv2.LINE_AA)

            if draw_landmarks:
                landmarks = face['landmarks']
                for landmark in landmarks:
                    radius = 5
                    thickness = -1
                    cv2.circle(image, landmark, radius, color, thickness, cv2.LINE_AA)

            if show_confidence:
                confidence = face['confidence']
                confidence = "{:.2f}".format(confidence)
                position = (face['x1'], face['y1'] - 10)
                font = cv2.FONT_HERSHEY_SIMPLEX
                scale = 0.5
                thickness = 2
                cv2.putText(image, confidence, position, font, scale, color, thickness, cv2.LINE_AA)
        return image

    def scale_coords(self, image, prediction):
        ih, iw = image.shape[:2]
        rw, rh = self.img_size
        a = np.array([
                (prediction['x1'], prediction['y1']),
                (prediction['x1'] + prediction['x2'], prediction['y1'] + prediction['y2'])
                    ])
        b = np.array([iw/rw, ih/rh])
        c = a * b
        prediction['img_width'] = iw
        prediction['img_height'] = ih
        prediction['x1'] = int(c[0,0].round())
        prediction['x2'] = int(c[1,0].round())
        prediction['y1'] = int(c[0,1].round())
        prediction['y2'] = int(c[1,1].round())
        prediction['face_width'] = (c[1,0] - c[0,0])
        prediction['face_height'] = (c[1,1] - c[0,1])
        # prediction['face_width'] = prediction['x2'] - prediction['x1']
        # prediction['face_height'] = prediction['y2'] - prediction['y1']
        prediction['area'] = prediction['face_width'] * prediction['face_height']
        prediction['pct_of_frame'] = prediction['area']/(prediction['img_width'] * prediction['img_height'])
        return prediction

    def detect(self, image):
        if isinstance(image, str):
            image = cv2.imread(str(image))
        img = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
        img = cv2.resize(img, self.img_size)
        self.fd.setInputSize(self.img_size)
        _, faces = self.fd.detect(img)
        if faces is None:
            return None
        else:
            predictions = self.parse_predictions(image, faces)
            return predictions

    def parse_predictions(self,
                          image,
                          faces):
        data = []
        for num, face in enumerate(list(faces)):
            x1, y1, x2, y2 = list(map(int, face[:4]))
            landmarks = list(map(int, face[4:len(face)-1]))
            landmarks = np.array_split(landmarks, len(landmarks) / 2)
            positions = ['left_eye', 'right_eye', 'nose', 'right_mouth', 'left_mouth']
            landmarks = {positions[num]: x.tolist() for num, x in enumerate(landmarks)}
            confidence = face[-1]
            datum = {'x1': x1,
                     'y1': y1,
                     'x2': x2,
                     'y2': y2,
                     'face_num': num,
                     'landmarks': landmarks,
                     'confidence': confidence,
                     'model': 'yunet'}
            d = self.scale_coords(image, datum)
            data.append(d)
        return data


In [7]:
import IPython
def show_image(image):
    _, ret = cv2.imencode('.jpg', image)
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

In [22]:
fd = FaceDetectorYunet()

In [ ]:
img = io.imread('https://github.com/astaileyyoung/CineFace/blob/main/research/notebooks/images/img_1.jpg?raw=true')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
faces = fd.detect(img)
if faces:
    fd.draw_faces(img, faces)
show_image(img)

In [ ]:
# img = io.imread('https://github.com/astaileyyoung/CineFace/blob/main/research/notebooks/images/img_2.jpg?raw=true')
img = io.imread('./data/img_2.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
faces = fd.detect(img)
if faces:
    fd.draw_faces(img, faces)
show_image(img)

In [ ]:
file_name = "data/636px-NASA_Astronaut_Group_18.jpg"
file_name = "data/img_2.jpg"

# img = cv2.imread(file_name, 0)
img = io.imread(file_name)
# img = io.imread('https://github.com/astaileyyoung/CineFace/blob/main/research/notebooks/images/img_2.jpg?raw=true')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

fd = FaceDetectorYunet()
faces = fd.detect(img)
if faces:
    fd.draw_faces(img, faces)
show_image(img)

In [ ]:
len(faces)

In [ ]:
import matplotlib.pyplot as plt



plt.figure(figsize=(20,10))
plt.imshow(img)
plt.axis('off')

Reference : https://medium.com/@yongsun.yoon/nba-face-recognition-system-345034ffed8c

In [1]:
import insightface
from insightface.app.common import Face
from insightface.model_zoo import model_zoo
import cv2
import os


det_model_path = 'buffalo_s/det_500m.onnx'
rec_model_path = 'buffalo_s/w600k_mbf.onnx'

det_model = model_zoo.get_model(f'./model/{det_model_path}')
rec_model = model_zoo.get_model(f'./model/{rec_model_path}')

det_model.prepare(ctx_id=0, input_size=(640, 640), det_thres=0.5)

/home/aiap/miniconda3/envs/cv/lib/python3.11/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/home/aiap/miniconda3/envs/cv/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


In [2]:
# file_name = ["data/636px-NASA_Astronaut_Group_18.jpg", "data/955px-NASA_Astronaut_Group_18.jpg","data/1432px-NASA_Astronaut_Group_18.jpg", "data/foreign-workers-5.jpg"]
# file_name = ["data/Ss.jpg", 'data/test.jpeg', 'data/jxjy.jpeg']
file_name = ['data/ref_data/lwk.jpg']
img = cv2.imread(file_name[0], cv2.IMREAD_UNCHANGED)
bboxes, kpss = det_model.detect(img, max_num=0, metric='default')

(218, 327, 3)


In [55]:
def draw_faces(image,
               faces,
               draw_landmarks=False,
               show_confidence=False,
               names=[],
               show_names=False):
    
    for i, face in enumerate(faces):
        color = (0, 255, 255)
        thickness = 2
        cv2.rectangle(image, (int(face[0]), int(face[1])), (int(face[2]), int(face[3])), color, thickness, cv2.LINE_AA)

        if draw_landmarks:
            landmarks = face['landmarks']
            for landmark in landmarks:
                radius = 5
                thickness = -1
                cv2.circle(image, landmark, radius, color, thickness, cv2.LINE_AA)

        if show_confidence:
            confidence = face[4]
            confidence = "{:.2f}".format(confidence)
            position = (int(face[0]), int(face[1] - 10))
            font = cv2.FONT_HERSHEY_SIMPLEX
            scale = 0.5
            thickness = 1
            cv2.putText(image, confidence, position, font, scale, color, thickness, cv2.LINE_AA)

        if show_names and len(faces)==len(names):

            position = (int(face[0]), int(face[1] - 25))
            font = cv2.FONT_HERSHEY_SIMPLEX
            scale = 0.5
            thickness = 1
            cv2.putText(image, names[i], position, font, scale, color, thickness, cv2.LINE_AA)            

    return image

In [ ]:
# cv2.putText(img=img,'Test',(100,60))

In [37]:
img = draw_faces(img,bboxes, show_confidence=True)

Get first bounding box and keypoints

In [ ]:
bbox = bboxes[0, :4]
confidence_score = bboxes[0, 4]
# keypoint
# det_score

bboxes[1,4]

In [ ]:
kpss[0]

In [ ]:

# face = bboxes[0]
color = (0, 0, 255)
# thickness = 2
# cv2.rectangle(img, (int(face[0]), int(face[1])), (int(face[2]), int(face[3])), color, thickness, cv2.LINE_AA)

cv2.circle(img=img, center=(201,116), radius=3, color=color, thickness=2)
cv2.circle(img=img, center=(231,118), radius=2, color=(0, 255, 0), thickness=2)
cv2.circle(img=img, center=(217, 134), radius=2, color=(0, 0, 255), thickness=2)

In [16]:
import IPython
def show_image(image):
    _, ret = cv2.imencode('.jpg', image)
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

In [ ]:
bboxes[0]

In [ ]:
show_image(img)

In [4]:
ref_data_dir = os.path.join('data', 'ref_data')
ref_files = [file for file in os.listdir(ref_data_dir) if os.path.isfile(os.path.join(ref_data_dir,file))]
# print(ref_files)
# file_name = os.path.join(ref_data_dir,ref_files[0])
# img = cv2.imread(file_name, cv2.IMREAD_COLOR)


In [5]:
import time
WIDTH = 320

In [ ]:
ref_embeddings, ref_names = [], []

for file in ref_files:

    file_path = os.path.join(ref_data_dir,file)
    img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)

    print(file.split('.'))
    print(img.shape)
    (h, w) = img.shape[:2]
    aspect_ratio = w / h

    new_height = int(WIDTH /aspect_ratio)

    # print(f'{new_height}')
    img = cv2.resize(img,(WIDTH, new_height))
    print(f'Resize shape: {img.shape}')

    bboxes, kpss = det_model.detect(img, max_num=0, metric='default')
    # img = draw_faces(img,bboxes, show_confidence=True)

    bbox = bboxes[0,:4]
    confidence_score = bboxes[0,4]

    kps = kpss[0]
    # print(f'bbox shape: {bbox.shape}')
    # print(f'kpss shape {kpss.shape}')

    face = Face(bbox=bbox, kps = kps, det_score =confidence_score)
    rec_model.get(img, face)

    ref_embeddings.append(face.normed_embedding)
    ref_names.append(file.split('.')[0])


    # show_image(img)
    # time.sleep(1)

The reference embeddings

In [ ]:
print(len(ref_embeddings))
print(ref_names)

In [51]:
file_name = ["data/Ss.jpg", 'data/test.jpeg']
img = cv2.imread(file_name[0], cv2.IMREAD_UNCHANGED)
bboxes, kpss = det_model.detect(img, max_num=0, metric='default')

In [41]:
bboxes.shape
img = draw_faces(img,bboxes, show_confidence=True)

In [52]:
def search (ref_embeddings, ref_names, unknown_embedding, threshold):
    score = np.dot(ref_embeddings, unknown_embedding)
    score = np.clip(score, 0, 1)
    idx = np.argmax(score, axis=0)
    id = 'unknown' # Set default id to unknown
    if score[idx] > threshold:
        id = ref_names[idx]
    return id


    

In [56]:
ids = []
for bbox, kps in zip(bboxes, kpss):
    confidence_score = bbox[4]
    bbox = bbox[0:4]
    face = Face(bbox=bbox, kps = kps, det_score =confidence_score)
    rec_model.get(img, face)
    # predict_face_embbedding = face.normed_embedding
    id =search(ref_embeddings=ref_embeddings,
           ref_names=ref_names,
           unknown_embedding=face.normed_embedding,
           threshold=0.5)
    ids.append(id)

    
img = draw_faces(img,bboxes, show_confidence=True, names=ids, show_names= True)

    

In [ ]:
ids